In [1]:
#Install Packages
# !pip install faiss-cpu
# !pip install sentence-transformers

DEPRECATION: Loading egg at c:\users\mansh\appdata\local\programs\python\python311\lib\site-packages\apache_beam-2.56.0-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\mansh\appdata\local\programs\python\python311\lib\site-packages\avro_python3-1.10.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\mansh\appdata\local\programs\python\python311\lib\site-packages\object_detection-0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loadi

DEPRECATION: Loading egg at c:\users\mansh\appdata\local\programs\python\python311\lib\site-packages\apache_beam-2.56.0-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\mansh\appdata\local\programs\python\python311\lib\site-packages\avro_python3-1.10.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\mansh\appdata\local\programs\python\python311\lib\site-packages\object_detection-0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loadi

In [2]:
import pandas as pd
df = pd.read_csv("sample_text.csv")
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps cont...",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mu...,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity i...,Travel


# Step 1 : Create source embeddings for the text column

In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [5]:
vectors.shape

(8, 768)

In [6]:
dim = vectors.shape[1]
dim

768

# Step 2 : Build a FAISS Index for vectors

In [7]:
import faiss

index = faiss.IndexFlatL2(dim)

# Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [8]:
index.add(vectors)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000019D51C65F50> >

# Step 4 : Encode search text using same encorder and normalize the output vector

In [9]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [10]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

# Step 5: Search for similar vector in the FAISS index created

In [11]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844836, 1.4039094]], dtype=float32)

In [12]:
I

array([[3, 2]], dtype=int64)

In [13]:
I.tolist()

[[3, 2]]

In [14]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [15]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [16]:
search_query

'I want to buy a polo t-shirt'

We can see that the two results from the dataframe are similar to a search_query